In [146]:
import os
from dotenv import load_dotenv
from datetime import datetime

load_dotenv()

BINANCE_API_KEY = os.getenv("BINANCE_API_KEY")
BINANCE_API_SECRET = os.getenv("BINANCE_API_SECRET")

In [147]:
type(datetime.today().strftime("%d %b, %Y"))

str

In [148]:
import requests
import pandas as pd
from binance.client import Client
import time

# --- Step 1: Fetch Top 50 Cryptocurrencies by Market Cap from CoinMarketCap ---
with open("data/top_50_cryptos.txt", "r") as f:
    top_50 = [line.strip() for line in f.readlines()]

print(f"Top 50 Cryptos Loaded: {top_50}")

# --- Step 2: Check Which Coins Are Tradable on Binance ---
API_KEY = BINANCE_API_KEY
API_SECRET = BINANCE_API_SECRET

client = Client(API_KEY, API_SECRET)

# Get all Binance trading pairs
exchange_info = client.get_exchange_info()
binance_symbols = {s["symbol"]: s for s in exchange_info["symbols"] if s["status"] == "TRADING"}

# Filter valid USDT trading pairs
valid_pairs = [f"{symbol}USDT" for symbol in top_50 if f"{symbol}USDT" in binance_symbols]
print(f"Tradable Cryptos on Binance: {valid_pairs}")

# --- Step 3: Fetch Historical Data for Valid Pairs ---
def get_historical_data(symbol, interval=Client.KLINE_INTERVAL_4HOUR, start_date="25 Mar, 2020", end_date=datetime.today().strftime("%d %b, %Y")):
    all_klines = []
    last_timestamp = client._get_earliest_valid_timestamp(symbol, interval)
    
    while True:
        klines = client.get_historical_klines(symbol, interval, start_date, end_date)

        if not klines:
            break  # Stop if no data is returned

        all_klines.extend(klines)
        last_timestamp = klines[-1][0]
        start_date = pd.to_datetime(last_timestamp, unit="ms").strftime("%d %b, %Y")
        time.sleep(1)  # Avoid hitting rate limits

    # Convert to DataFrame
    df = pd.DataFrame(all_klines, columns=["Open Time", "Open", "High", "Low", "Close", "Volume",
                                           "Close Time", "Quote Asset Volume", "Number of Trades",
                                           "Taker Buy Base", "Taker Buy Quote", "Ignore"])

    df["Open Time"] = pd.to_datetime(df["Open Time"], unit="ms")
    df["Close Time"] = pd.to_datetime(df["Close Time"], unit="ms")
    df[["Open", "High", "Low", "Close", "Volume"]] = df[["Open", "High", "Low", "Close", "Volume"]].astype(float)
    
    return df

# Dictionary to store historical data for valid pairs
crypto_data = {}

for pair in valid_pairs:
    print(f"Fetching historical data for {pair}...")
    crypto_data[pair] = get_historical_data(pair)

# --- Step 4: Merge Data into a Single DataFrame for Model Training ---
combined_df = pd.DataFrame()

for symbol, df in crypto_data.items():
    df = df[["Open Time", "Close", "Volume"]].copy()
    df.rename(columns={"Close": f"{symbol}_Close", "Volume": f"{symbol}_Volume"}, inplace=True)
    
    if combined_df.empty:
        combined_df = df
    else:
        combined_df = combined_df.merge(df, on="Open Time", how="outer")

combined_df.fillna(method="ffill", inplace=True)

# Save to CSV (optional)
combined_df.to_csv("data/crypto_historical_data.csv", index=False)
print("Data successfully saved!")

Top 50 Cryptos Loaded: ['BTC', 'ETH', 'USDT', 'XRP', 'SOL', 'BNB', 'USDC', 'DOGE', 'ADA', 'TRX', 'LINK', 'AVAX', 'SUI', 'XLM', 'TON', 'SHIB', 'LEO', 'LTC', 'HBAR', 'HYPE', 'DOT', 'BGB', 'BCH', 'USDe', 'OM', 'UNI', 'DAI', 'ONDO', 'XMR', 'PEPE', 'NEAR', 'AAVE', 'MNT', 'ICP', 'APT', 'TAO', 'TRUMP', 'ETC', 'OKB', 'VET', 'POL', 'KAS', 'CRO', 'ALGO', 'RENDER', 'JUP', 'FIL', 'ARB', 'GT', 'FDUSD']
Tradable Cryptos on Binance: ['BTCUSDT', 'ETHUSDT', 'XRPUSDT', 'SOLUSDT', 'BNBUSDT', 'USDCUSDT', 'DOGEUSDT', 'ADAUSDT', 'TRXUSDT', 'LINKUSDT', 'AVAXUSDT', 'SUIUSDT', 'XLMUSDT', 'TONUSDT', 'SHIBUSDT', 'LTCUSDT', 'HBARUSDT', 'DOTUSDT', 'BCHUSDT', 'OMUSDT', 'UNIUSDT', 'PEPEUSDT', 'NEARUSDT', 'AAVEUSDT', 'ICPUSDT', 'APTUSDT', 'TAOUSDT', 'TRUMPUSDT', 'ETCUSDT', 'VETUSDT', 'POLUSDT', 'ALGOUSDT', 'RENDERUSDT', 'JUPUSDT', 'FILUSDT', 'ARBUSDT', 'FDUSDUSDT']
Fetching historical data for BTCUSDT...
Fetching historical data for ETHUSDT...
Fetching historical data for XRPUSDT...
Fetching historical data for SOLUS

/var/folders/sh/s_xv2d8x6kv1_qyc9qgmkv300000gn/T/ipykernel_98676/3364079926.py:72: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  combined_df.fillna(method="ffill", inplace=True)


Data successfully saved!


In [149]:
combined_df

,Open Time,BTCUSDT_Close,BTCUSDT_Volume,ETHUSDT_Close,ETHUSDT_Volume,XRPUSDT_Close,XRPUSDT_Volume,SOLUSDT_Close,SOLUSDT_Volume,BNBUSDT_Close,...,RENDERUSDT_Close,RENDERUSDT_Volume,JUPUSDT_Close,JUPUSDT_Volume,FILUSDT_Close,FILUSDT_Volume,ARBUSDT_Close,ARBUSDT_Volume,FDUSDUSDT_Close,FDUSDUSDT_Volume
0,2020-03-25 00:00:00,6557.75,17042.837312,136.09,184453.53893,0.16024,34368839.9,NaN,NaN,12.1749,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-03-25 04:00:00,6675.21,12406.070557,138.27,122363.50920,0.16226,24748020.1,NaN,NaN,12.4163,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-03-25 08:00:00,6521.36,45370.620679,134.37,343216.98456,0.15810,64665116.6,NaN,NaN,12.1400,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-03-25 12:00:00,6617.27,30706.295240,135.47,196526.13761,0.15938,41978907.8,NaN,NaN,12.2659,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-03-25 16:00:00,6622.38,14683.351661,135.77,92691.18497,0.16026,18471806.2,NaN,NaN,12.2300,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10706,2025-02-11 08:00:00,98151.52,1703.700260,2708.80,59949.36250,2.49640,40634576.0,203.40,683735.412,643.4100,...,4.695,952262.22,0.8608,3017638.4,3.490,762150.77,0.4839,8442193.1,0.9992,73456491.0
10707,2025-02-11 12:00:00,97007.53,4245.102450,2662.36,87537.49870,2.46970,40728574.0,199.47,729499.570,637.7100,...,4.633,606014.63,0.8356,30992993.5,3.408,1425235.56,0.4716,15594434.6,0.9989,121175559.0
10708,2025-02-11 16:00:00,95040.82,4884.737110,2589.70,103602.58910,2.38750,52993364.0,195.65,410134.514,628.4700,...,4.378,1017173.78,0.8135,7518637.1,3.287,983450.95,0.4597,10146617.1,0.9990,66517972.0
10709,2025-02-11 20:00:00,95778.20,3202.620870,2602.59,101923.08320,2.41310,33292005.0,197.96,390879.150,642.1100,...,4.458,568676.61,0.8241,2242013.3,3.339,551669.25,0.4629,5984383.3,0.9990,65966085.0


In [150]:
# Apply Indicators to Each Cryptocurrency
from numpy import nan as npNaN
import pandas_ta as ta

ta_df = combined_df.copy()

for col in combined_df.columns:
    if col.endswith("_Close"):  
        symbol = col.replace("_Close", "")

        # Ensure numeric inputs are float
        close_price = ta_df[col].astype(float)
        volume = ta_df[f"{symbol}_Volume"].astype(float)

        # Trend Indicators
        ta_df[f"{symbol}_EMA_10"] = ta.ema(close_price, length=10).astype(float)
        ta_df[f"{symbol}_EMA_50"] = ta.ema(close_price, length=50).astype(float)

        # Momentum Indicators
        ta_df[f"{symbol}_RSI_14"] = ta.rsi(close_price, length=14).astype(float)
        macd = ta.macd(close_price)
        ta_df[f"{symbol}_MACD"] = macd["MACD_12_26_9"].astype(float)
        ta_df[f"{symbol}_MACD_Signal"] = macd["MACDs_12_26_9"].astype(float)

        # Volatility Indicators
        bb = ta.bbands(close_price, length=20)
        ta_df[f"{symbol}_BB_Upper"] = bb["BBU_20_2.0"].astype(float)
        ta_df[f"{symbol}_BB_Middle"] = bb["BBM_20_2.0"].astype(float)

        # Volume Indicators
        ta_df[f"{symbol}_OBV"] = ta.obv(close_price, volume).astype(float)
        ta_df[f"{symbol}_MFI_14"] = ta.mfi(close_price, close_price, close_price, volume, length=14).astype(float)

# Save to CSV
combined_df.to_csv("data/crypto_with_indicators.csv", index=False)
print("Technical indicators added successfully!")



/var/folders/sh/s_xv2d8x6kv1_qyc9qgmkv300000gn/T/ipykernel_98676/954176369.py:32: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[8.28131262e+07 2.03191846e+08 9.72387344e+07 ... 1.97647147e+08
 2.54190510e+08 3.06741262e+08]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  ta_df[f"{symbol}_MFI_14"] = ta.mfi(close_price, close_price, close_price, volume, length=14).astype(float)
/var/folders/sh/s_xv2d8x6kv1_qyc9qgmkv300000gn/T/ipykernel_98676/954176369.py:32: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[2.95878151e+08 8.76178562e+07 9.87543554e+07 ... 4.11806903e+08
 4.64249420e+08 2.29415966e+08]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  ta_df[f"{symbol}_MFI_14"] = ta.mfi(close_price, close_price, close_price, volume, length=14).asty

Technical indicators added successfully!


In [151]:
ta_df

,Open Time,BTCUSDT_Close,BTCUSDT_Volume,ETHUSDT_Close,ETHUSDT_Volume,XRPUSDT_Close,XRPUSDT_Volume,SOLUSDT_Close,SOLUSDT_Volume,BNBUSDT_Close,...,ARBUSDT_MFI_14,FDUSDUSDT_EMA_10,FDUSDUSDT_EMA_50,FDUSDUSDT_RSI_14,FDUSDUSDT_MACD,FDUSDUSDT_MACD_Signal,FDUSDUSDT_BB_Upper,FDUSDUSDT_BB_Middle,FDUSDUSDT_OBV,FDUSDUSDT_MFI_14
0,2020-03-25 00:00:00,6557.75,17042.837312,136.09,184453.53893,0.16024,34368839.9,NaN,NaN,12.1749,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-03-25 04:00:00,6675.21,12406.070557,138.27,122363.50920,0.16226,24748020.1,NaN,NaN,12.4163,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-03-25 08:00:00,6521.36,45370.620679,134.37,343216.98456,0.15810,64665116.6,NaN,NaN,12.1400,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-03-25 12:00:00,6617.27,30706.295240,135.47,196526.13761,0.15938,41978907.8,NaN,NaN,12.2659,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-03-25 16:00:00,6622.38,14683.351661,135.77,92691.18497,0.16026,18471806.2,NaN,NaN,12.2300,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10706,2025-02-11 08:00:00,98151.52,1703.700260,2708.80,59949.36250,2.49640,40634576.0,203.40,683735.412,643.4100,...,72.791186,0.999053,0.999060,57.505083,0.000024,-6.462698e-06,0.999213,0.998980,-3.183055e+09,69.372339
10707,2025-02-11 12:00:00,97007.53,4245.102450,2662.36,87537.49870,2.46970,40728574.0,199.47,729499.570,637.7100,...,67.855502,0.999025,0.999054,46.101294,0.000011,-3.055951e-06,0.999211,0.998975,-3.304231e+09,52.088608
10708,2025-02-11 16:00:00,95040.82,4884.737110,2589.70,103602.58910,2.38750,52993364.0,195.65,410134.514,628.4700,...,66.089645,0.999020,0.999052,49.683240,0.000008,-7.954185e-07,0.999189,0.998990,-3.237713e+09,62.617354
10709,2025-02-11 20:00:00,95778.20,3202.620870,2602.59,101923.08320,2.41310,33292005.0,197.96,390879.150,642.1100,...,71.329168,0.999017,0.999050,49.683240,0.000006,6.300015e-07,0.999190,0.998995,-3.237713e+09,58.344835


In [152]:
ta_df['Open Time']

0       2020-03-25 00:00:00
1       2020-03-25 04:00:00
2       2020-03-25 08:00:00
3       2020-03-25 12:00:00
4       2020-03-25 16:00:00
                ...        
10706   2025-02-11 08:00:00
10707   2025-02-11 12:00:00
10708   2025-02-11 16:00:00
10709   2025-02-11 20:00:00
10710   2025-02-12 00:00:00
Name: Open Time, Length: 10711, dtype: datetime64[ns]

In [153]:
START_TIME = int(ta_df["Open Time"].min().timestamp() * 1000)  # To be set based on `ta_df`
END_TIME = int(ta_df["Open Time"].max().timestamp() * 1000)   # Current timestamp

In [154]:
def get_historical_funding_rates(symbol, lookback_days=365):  # Default to 1 year
    try:
        end_time = int(ta_df["Open Time"].max().timestamp() * 1000)  # Current timestamp in milliseconds
        start_time = int(ta_df["Open Time"].min().timestamp() * 1000)  # Go back `lookback_days`

        all_funding_rates = {}

        while start_time < end_time:
            funding_data = client.futures_funding_rate(symbol=symbol, startTime=start_time, endTime=end_time, limit=1000)
            
            if not funding_data:
                break  # Stop if there's no more data
            
            for entry in funding_data:
                all_funding_rates[entry["fundingTime"]] = float(entry["fundingRate"])

            # Move start_time forward to fetch the next batch
            start_time = funding_data[-1]["fundingTime"] + 1  # Avoid duplicate timestamps

        return all_funding_rates
    except Exception as e:
        print(f"Error fetching historical Funding Rates for {symbol}: {e}")
        return None

In [155]:
# Copy ta_df to futures_df to store futures data separately
futures_df = ta_df.copy()

# Loop through all assets in `ta_df` and fetch only funding rates
lookback_days = 365  # Fetch 1 year of funding data

for col in futures_df.columns:
    if col.endswith("_Close"):  
        symbol = col.replace("_Close", "")
        futures_symbol = f"{symbol}USDT"  # Binance Futures symbol format

        # Fetch historical funding rates
        historical_funding = get_historical_funding_rates(futures_symbol, lookback_days=lookback_days)

        # Convert to DataFrame and merge with `futures_df`
        if historical_funding:
            funding_df = pd.DataFrame.from_dict(historical_funding, orient="index", columns=[f"{symbol}_Funding_Rate"])
            funding_df.index = pd.to_datetime(funding_df.index, unit="ms")  # Convert timestamp
            futures_df = futures_df.merge(funding_df, left_on="Open Time", right_index=True, how="left")

print("✅ Historical Funding Rates successfully added to `futures_df`!")


✅ Historical Funding Rates successfully added to `futures_df`!


In [156]:
futures_df

,Open Time,BTCUSDT_Close,BTCUSDT_Volume,ETHUSDT_Close,ETHUSDT_Volume,XRPUSDT_Close,XRPUSDT_Volume,SOLUSDT_Close,SOLUSDT_Volume,BNBUSDT_Close,...,ARBUSDT_MFI_14,FDUSDUSDT_EMA_10,FDUSDUSDT_EMA_50,FDUSDUSDT_RSI_14,FDUSDUSDT_MACD,FDUSDUSDT_MACD_Signal,FDUSDUSDT_BB_Upper,FDUSDUSDT_BB_Middle,FDUSDUSDT_OBV,FDUSDUSDT_MFI_14
0,2020-03-25 00:00:00,6557.75,17042.837312,136.09,184453.53893,0.16024,34368839.9,NaN,NaN,12.1749,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-03-25 04:00:00,6675.21,12406.070557,138.27,122363.50920,0.16226,24748020.1,NaN,NaN,12.4163,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-03-25 08:00:00,6521.36,45370.620679,134.37,343216.98456,0.15810,64665116.6,NaN,NaN,12.1400,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-03-25 12:00:00,6617.27,30706.295240,135.47,196526.13761,0.15938,41978907.8,NaN,NaN,12.2659,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-03-25 16:00:00,6622.38,14683.351661,135.77,92691.18497,0.16026,18471806.2,NaN,NaN,12.2300,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10706,2025-02-11 08:00:00,98151.52,1703.700260,2708.80,59949.36250,2.49640,40634576.0,203.40,683735.412,643.4100,...,72.791186,0.999053,0.999060,57.505083,0.000024,-6.462698e-06,0.999213,0.998980,-3.183055e+09,69.372339
10707,2025-02-11 12:00:00,97007.53,4245.102450,2662.36,87537.49870,2.46970,40728574.0,199.47,729499.570,637.7100,...,67.855502,0.999025,0.999054,46.101294,0.000011,-3.055951e-06,0.999211,0.998975,-3.304231e+09,52.088608
10708,2025-02-11 16:00:00,95040.82,4884.737110,2589.70,103602.58910,2.38750,52993364.0,195.65,410134.514,628.4700,...,66.089645,0.999020,0.999052,49.683240,0.000008,-7.954185e-07,0.999189,0.998990,-3.237713e+09,62.617354
10709,2025-02-11 20:00:00,95778.20,3202.620870,2602.59,101923.08320,2.41310,33292005.0,197.96,390879.150,642.1100,...,71.329168,0.999017,0.999050,49.683240,0.000006,6.300015e-07,0.999190,0.998995,-3.237713e+09,58.344835
